# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [ ]:
# ! pip install -q swig
# ! pip install -q gymnasium[box2d,classic_control]

zsh:1: no matches found: gymnasium[box2d,classic_control]


## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [ ]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

: 

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
import numpy as np

def random_agent_function(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

/Users/nguyentan/Documents/CSTTNT_NC/lab02_NguyenDuyTan_3121410443/lab_env/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
2025-09-27 12:07:37.891 Python[10941:1785193] +[IMKClient subclass]: chose IMKClient_Legacy
2025-09-27 12:07:37.891 Python[10941:1785193] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Obs: [-0.0038085   1.4092884  -0.3857592  -0.07252411  0.00441973  0.08738023
  0.          0.        ] -> Action: 1
Obs: [-0.0076849   1.407072   -0.39373142 -0.09853994  0.01044154  0.12044845
  0.          0.        ] -> Action: 2
Obs: [-0.01171064  1.4049569  -0.40796328 -0.09404357  0.01576674  0.10651366
  0.          0.        ] -> Action: 1
Obs: [-0.01582413  1.4022324  -0.41897193 -0.12119433  0.02329921  0.15066363
  0.          0.        ] -> Action: 2
Obs: [-0.01978369  1.400344   -0.40442163 -0.08408438  0.03167401  0.16751179
  0.          0.        ] -> Action: 1
Obs: [-0.02383966  1.3978423  -0.41651997 -0.11144965  0.04247528  0.2160453
  0.          0.        ] -> Action: 1
Obs: [-0.02796431  1.394742   -0.4251047  -0.13819529  0.05498829  0.25028336
  0.          0.        ] -> Action: 3
Obs: [-0.03200979  1.3910593  -0.41514364 -0.16409582  0.06548133  0.20987976
  0.          0.        ] -> Action: 2
Obs: [-0.03602419  1.3879848  -0.41242152 -0.13715455  0.07637374

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [ ]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [ ]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent_function)

Obs: [ 9.8676677e-04  1.4073550e+00  9.9933580e-02 -1.5844883e-01
 -1.1366399e-03 -2.2636455e-02  0.0000000e+00  0.0000000e+00] -> Action: 0
Obs: [ 0.00197344  1.4032124   0.09979869 -0.18410744 -0.00225527 -0.02237392
  0.          0.        ] -> Action: 0
Obs: [ 0.00296021  1.3984699   0.09980219 -0.21078129 -0.00337316 -0.02236013
  0.          0.        ] -> Action: 0
Obs: [ 0.00394688  1.3931273   0.09980547 -0.23745337 -0.0044909  -0.02235704
  0.          0.        ] -> Action: 0
Obs: [ 0.00493364  1.3871846   0.0998088  -0.26412204 -0.00560848 -0.0223535
  0.          0.        ] -> Action: 0
Obs: [ 0.00592051  1.3806419   0.09981211 -0.29079035 -0.00672587 -0.02234988
  0.          0.        ] -> Action: 0
Obs: [ 0.00690727  1.3734993   0.09981541 -0.31745875 -0.00784311 -0.02234684
  0.          0.        ] -> Action: 2
Obs: [ 0.00777426  1.3663726   0.08842586 -0.31675628 -0.00954652 -0.03407148
  0.          0.        ] -> Action: 2
Obs: [ 0.00853224  1.3597461   0.0780869 

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [ ]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
def improved_rocket_agent_function(observation):
    """
    A better reflex-based agent that uses all available thrusters.
    
    Controls:
    - Main thruster: Slow down descent when falling too fast
    - Left/Right thrusters: Control horizontal position, angle, and angular velocity
    """
    
    # Extract observation values
    x = observation[Obs.X.value]           # Horizontal position
    y = observation[Obs.Y.value]           # Vertical position  
    vx = observation[Obs.VX.value]         # Horizontal velocity
    vy = observation[Obs.VY.value]         # Vertical velocity
    angle = observation[Obs.ANGLE.value]   # Angle (0 = upright)
    angular_vel = observation[Obs.ANGULAR_VELOCITY.value]  # Angular velocity
    
    # Priority 1: Control descent speed with main thruster
    # Fire main thruster if falling too fast or too close to ground with high speed
    if vy < -0.4 or (y < 0.3 and vy < -0.2):
        return Act.MAIN.value
    
    # Priority 2: Control angle - keep lander upright
    # If tilted significantly or rotating fast, use thrusters to stabilize
    if angle > 0.2 or angular_vel > 0.3:  # Tilted right or rotating right
        return Act.LEFT.value  # Fire left thruster to rotate left
    elif angle < -0.2 or angular_vel < -0.3:  # Tilted left or rotating left  
        return Act.RIGHT.value  # Fire right thruster to rotate right
    
    # Priority 3: Control horizontal position - move toward center (x=0)
    # If moving away from center or too far from center, correct
    if x > 0.1 and vx > -0.1:  # Too far right or not moving left fast enough
        return Act.LEFT.value   # Fire left thruster to push right
    elif x < -0.1 and vx < 0.1:  # Too far left or not moving right fast enough
        return Act.RIGHT.value  # Fire right thruster to push left
    
    # Priority 4: Control horizontal velocity - slow down horizontal movement
    if abs(vx) > 0.3:  # Moving too fast horizontally
        if vx > 0:  # Moving right too fast
            return Act.LEFT.value   # Fire left thruster to slow down
        else:  # Moving left too fast
            return Act.RIGHT.value  # Fire right thruster to slow down
    
    # If everything looks good, do nothing
    return Act.NO_OP.value

# Test the improved agent
print("Testing improved agent with single episode:")
run_episode(improved_rocket_agent_function)

Testing improved agent with single episode:
Obs: [ 0.00510616  1.4070227   0.51718765 -0.1732331  -0.00591002 -0.11715093
  0.          0.        ] -> Action: 1
Obs: [ 0.01012144  1.4025514   0.5050523  -0.1987401  -0.00940504 -0.06990615
  0.          0.        ] -> Action: 1
Obs: [ 0.01503954  1.3974907   0.49286485 -0.22493432 -0.01044999 -0.0209005
  0.          0.        ] -> Action: 1
Obs: [ 0.01987648  1.3918341   0.48268574 -0.25138697 -0.00945315  0.01993883
  0.          0.        ] -> Action: 1
Obs: [ 0.02465372  1.3855894   0.47519946 -0.277531   -0.00695467  0.04997434
  0.          0.        ] -> Action: 1
Obs: [ 0.02934494  1.378756    0.4644126  -0.30369717 -0.00229396  0.09322266
  0.          0.        ] -> Action: 1
Obs: [ 0.03394604  1.3713146   0.45310083 -0.33073702  0.00463021  0.13849634
  0.          0.        ] -> Action: 1
Obs: [ 0.03847485  1.3632789   0.44404444 -0.3571938   0.01336374  0.17468673
  0.          0.        ] -> Action: 1
Obs: [ 0.04292364  1.

-100

In [ ]:
# Evaluate the improved agent performance
print("Evaluating improved agent over 100 episodes...")
improved_rewards = run_episodes(improved_rocket_agent_function, n=100)

print(f"Improved agent results:")
print(f"Average reward: {np.average(improved_rewards)}")
print(f"Success rate: {np.sum(np.array(improved_rewards) == 100)}/{len(improved_rewards)}")
print(f"Success percentage: {np.sum(np.array(improved_rewards) == 100)/len(improved_rewards)*100:.1f}%")

# Compare with original agent
print(f"\nComparison:")
print(f"Original agent success rate: 8/1000 (0.8%)")
print(f"Improved agent success rate: {np.sum(np.array(improved_rewards) == 100)}/{len(improved_rewards)} ({np.sum(np.array(improved_rewards) == 100)/len(improved_rewards)*100:.1f}%)")
print(f"Improvement factor: {(np.sum(np.array(improved_rewards) == 100)/len(improved_rewards))/(8/1000):.1f}x")

Evaluating improved agent over 100 episodes...
Improved agent results:
Average reward: -100.0
Success rate: 0/100
Success percentage: 0.0%

Comparison:
Original agent success rate: 8/1000 (0.8%)
Improved agent success rate: 0/100 (0.0%)
Improvement factor: 0.0x


: 